In [1]:
# calculate alchpot for every cube file along axis
# plot \mu vs shift

In [2]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import alchemy_tools2 as at
from parse_density_files import CUBE
import glob
import numpy as np
from matplotlib import pyplot as plt

In [5]:
cube_paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/shift_molecule/shift_*')
cube_paths.sort()
# cube_paths

### calculate alchemical potentials

In [4]:
for path in cube_paths[0:3]:
    cube = CUBE(path + '/DENSITY.cube')
    density = cube.data_scaled
    meshgrid = cube.get_grid()
    h_mat = cube.get_hmatrix()
    nuclei = cube.atoms
    ae_rep, ae, alch_pots = at.calculate_atomic_energies(density, nuclei, meshgrid, h_mat)
    
    save = np.array([nuclei, alch_pots]).T
    header = 'charge\t x_coord\t y_coord\t z_coord\t alchemical_potential'
    save_dir = os.path.join(path, 'alchpots.txt')
    np.savetxt(save_dir, store, delimiter='\t', header = header)

AttributeError: 'CUBE' object has no attribute 'get_hmatrix'

### plot results

In [7]:
c = CUBE(path + '/DENSITY.cube')

In [8]:
c.get_hmatrix()

AttributeError: 'CUBE' object has no attribute 'get_hmatrix'

In [9]:
class CUBE(object):
    
    def __init__(self, fname):
        f = open(fname, 'r')
        for i in range(2): f.readline() # echo comment
        tkns = f.readline().split() # number of atoms included in the file followed by the position of the origin of the volumetric data
        self.natoms = int(tkns[0])
        self.origin = np.array([float(tkns[1]),float(tkns[2]),float(tkns[3])])
        tkns = f.readline().split() #
        self.NX = int(tkns[0])
        self.X = np.array([float(tkns[1]),float(tkns[2]),float(tkns[3])])
        tkns = f.readline().split() #
        self.NY = int(tkns[0])
        self.Y = np.array([float(tkns[1]),float(tkns[2]),float(tkns[3])])
        tkns = f.readline().split() #
        self.NZ = int(tkns[0])
        self.Z = np.array([float(tkns[1]),float(tkns[2]),float(tkns[3])])
        
        self.dv = np.linalg.det(np.array([self.X, self.Y, self.Z])) # volume per gridpoint
        
        self.atoms = []
        for i in range(self.natoms):
          tkns = f.readline().split()
          self.atoms.append([float(tkns[0]), float(tkns[2]), float(tkns[3]), float(tkns[4])])
          
        self.atoms = np.array(self.atoms)
        
        self.data = np.zeros((self.NX,self.NY,self.NZ))
        i=0
        for s in f:
          for v in s.split():
            self.data[i//(self.NY*self.NZ), (i//self.NZ)%self.NY, i%self.NZ] = float(v)
            i+=1
        if i != self.NX*self.NY*self.NZ: raise NameError("FSCK!")
        
        self.scale() # values scaled by volume
        
    def project(self, axes):
        """
        scales density by gridvolume and projects density on specified axes (1D or 2D)
        """
        projected_density = np.sum(self.data*self.dv, axis=axes)
        return(projected_density)
        
    def get_axis(self, axis, unit='Ang'):
        """
        returns cell coordinates in Angstrom or Bohr
        """
        if axis == 'x' or axis == 0:
            l_x = self.X[0]*self.NX
            coords = np.linspace(self.origin[0], l_x-self.X[0], self.NX)
            
        if axis == 'y' or axis == 1:
            l_y = self.Y[1]*self.NY
            coords = np.linspace(self.origin[1], l_y-self.Y[1], self.NY)
            
        if axis == 'z' or axis == 2:
            l_z = self.Z[2]*self.NZ
            coords = np.linspace(self.origin[2], l_z-self.Z[2], self.NZ)
            
        if unit=='Ang':
            coords *= Bohr
        elif unit=='Bohr':
            pass
        else:
            raise Exception('Unknown unit')
            
        return(coords)
        
        
    def get_grid(self):
        """
        returns the coordinates of the grid points where the density values are given as a meshgrid
        works so far only for orthogonal coordinate axes
        """
        # length along the axes
        l_x = self.X[0]*self.NX
        l_y = self.Y[1]*self.NY
        l_z = self.Z[2]*self.NZ
        # gpts along every axis
        x_coords = np.linspace(self.origin[0], l_x-self.X[0], self.NX)
        y_coords = np.linspace(self.origin[1], l_y-self.Y[1], self.NY)
        z_coords = np.linspace(self.origin[2], l_z-self.Z[2], self.NZ)
        # create gridpoints
        meshgrid = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')
        return(meshgrid)
    
    def get_hmatrix(self):
        h_x = self.X*self.NX
        h_y = self.Y*self.NY
        h_z = self.Z*self.NZ
        return(np.array([h_x, h_y, h_z]))
        
        
    def scale(self):
        """
        calculate density scaled by volume of gridpoints
        """
        self.data_scaled = (self.data*self.dv).copy()
        
    def plot_projection(self, axes):
        """
        plot scaled projection of density along specified projection axis (1D, 2D)
        """
        projected_density = self.project(axes)
        
        if type(axes) == tuple:
            coordinate = np.linspace(self.origin[0], self.X[0]*self.NX*Bohr, self.NX)
            fig, ax = plt.subplots(1,1)
            ax.plot(coordinate, projected_density)
            ax.set_xlabel(r'Cell coordinate $x_0$ (Ang)')
            ax.set_ylabel(r'$\rho (x_0)$')
            
        if type(axes) == int:
            coordinate0 = np.linspace(self.origin[0]+0.5, self.X[0]*self.NX*Bohr-0.5, self.NX)
            coordinate1 = np.linspace(self.origin[0], self.X[0]*self.NX*Bohr, self.NX)
            fig, ax = plt.subplots(1,1)
            ax.contour(coordinate0, coordinate1, projected_density)